# College Tour Information Scraper

Gathers information from the https://www.youvisit.com/collegesearch/ website



In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [2]:
DRIVER_PATH = "./chromedriver"
WEBSITE_URL = "https://www.youvisit.com/collegesearch/"

In [3]:
def wait_for_element(xpath, seconds=10):
    try:
        item = WebDriverWait(driver, seconds).until(
            EC.presence_of_element_located((By.XPATH, xpath)))
    except exception:
        raise Exception("Could not find the specified xpath")
        
    return item

In [13]:
driver = webdriver.Chrome(DRIVER_PATH)
driver.get(WEBSITE_URL)
account_button_xpath = r"//*[@id='yv.com-cs-root']/div[1]/div[2]/div[1]/div[2]/button"
account_button_element = wait_for_element(account_button_xpath, seconds=10)
account_button_element.click()

In [14]:
email_button_xpath = r"/html/body/div[2]/div[1]/div/div/div/div/div/div/div/div/div[2]/div[6]/button"
email_button_element = wait_for_element(email_button_xpath, seconds=10)
email_button_element.click()

In [26]:
email_input_xpath = r"/html/body/div[1]/div[2]/div/div[1]/div[2]/div[2]/div[1]/input"
email_input_element = wait_for_element(email_button_xpath, seconds=10)
email_input_element.send_keys("email@gmail.com")
email_input_element.send_keys(Keys.ENTER)